# Keyword classification : high , medium and low suicide rate countries

#### Use of precomputed word embedding 

## Insight  1:

#### 

## Question 2:

#### Find the funtion to retrieve the created vocabulary ( reference: https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization). Retrieve vocabulary for 2 Ngram model and display and count how many times you see a phrase that contains word 'terrible'.  

## Prepare Keywords data

In [2]:
import os, pathlib, shutil, random
import tensorflow as tf
from tensorflow import keras

batch_size = 32
train_ds = keras.utils.text_dataset_from_directory("kwdsuic/kwd_per_paper/train", batch_size=batch_size, label_mode="categorical", shuffle=True,
                                                  labels="inferred", class_names=["C2","C4","C5","C6","C7","C8","C9","C10"] )

#val_ds = keras.utils.text_dataset_from_directory("kwdsuic/train", batch_size=batch_size, label_mode="categorical")
#test_ds = keras.utils.text_dataset_from_directory("kwdsuic/train", batch_size=batch_size, label_mode="categorical")

Found 29043 files belonging to 8 classes.


#### Inspect the dataset read from directory files 
Displaying the shapes and dtypes of the first batch

In [3]:
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    #print("inputs:", inputs)
    #print("targets:", targets)
    break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32, 8)
targets.dtype: <dtype: 'float32'>
inputs[0]: tf.Tensor(b'physician_assisted_dying euthanasia_act end_of_life_decision_making palliative_care', shape=(), dtype=string)
targets[0]: tf.Tensor([0. 0. 0. 0. 0. 1. 0. 0.], shape=(8,), dtype=float32)


#### Keep only text in text_only_train_ds. Use only text data. 

In [4]:
# This function returns only data part without target 
# to create a new dataset that will be used to create dictionary
text_only_train_ds = train_ds.map(lambda x, y: x)
for inputs in text_only_train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("inputs[0]:", inputs[0])
    break


inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
inputs[0]: tf.Tensor(b'urban_jail inmates', shape=(), dtype=string)


In [5]:
from tensorflow.keras.layers import TextVectorization
max_tokens=30000
# Text vectorization layer creates a structure that will populate with textual data
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization
# This layer performs text standardization, splitting it into Ngrams - single or pairs of words
# It prepares output according to output mode
text_vectorization = TextVectorization( ngrams=1, max_tokens=max_tokens, output_mode="count")

# Here vocabulary is created
text_vectorization.adapt(text_only_train_ds)

# Here datasets are created where the text in the string is converted
# All datasets are train dataset
binary_1gram_train_ds = train_ds.map( lambda x, y: (text_vectorization(x), y ), num_parallel_calls=4)
binary_1gram_val_ds = train_ds.map( lambda x, y: (text_vectorization(x), y),num_parallel_calls=4)
binary_1gram_test_ds = train_ds.map( lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)


In [20]:
for inputs, targets in binary_1gram_train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[:20]:", inputs[:20])
    print("targets[:20]:", targets[:20])
    break

inputs.shape: (32, 23788)
inputs.dtype: <dtype: 'float32'>
targets.shape: (32, 8)
targets.dtype: <dtype: 'float32'>
inputs[:20]: tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(20, 23788), dtype=float32)
targets[:20]: tf.Tensor(
[[0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0. 0. 0.]], shape=(20, 8), dtype=float32)


In [8]:
vc=text_vectorization.get_vocabulary()
for tg in vc:
    if 'sexual' in tg:
        print(tg)

print(len(vc))

sexualabuse
sexualorientation
childhoodsexualabuse
sexualviolence
sexualminority
sexualbehavior
homosexuality
sexuality
militarysexualtrauma
bisexualmen
bisexual
sexualassault
childsexualabuse
bisexualadults
sexualhealth
bisexualyouth
sexualtrauma
transsexual
sexuallytransmittedinfections
sexualminorityyouth
sexuallytransmitteddiseases
sexualandgenderminorities
sexualbehaviors
bisexualyouths
sexualminorities
sexualharassment
transsexuals
sexualidentity
sexualdysfunction
sexualandreproductivehealthandrights
hypersexuality
bisexualpopulations
bisexuality
transsexuality
transsexualism
sexualrisktaking
sexualriskbehaviors
sexualriskbehavior
sexualrisk
sexuallytransmitteddisease
riskysexualbehaviors
homosexualmen
homosexual
heterosexualyouth
bisexuals
adolescentsexualorientation
sociosexuality
sexualvictimization
sexualstigma
sexualorientationdisparities
sexualintercourse
sexualhomicide
sexualhealthpromotion
sexualcompulsivity
sexualbehaviour
sexualaddiction
sexualactivity
sexsexualpartners

#### This is function creates model from Listing 11.5
Our model-building utility

In [24]:
from tensorflow import keras
from tensorflow.keras import layers

# This is dense model , no embedding 
def get_model(max_tokens=max_tokens, hidden_dim=16):
    inputs = keras.Input(shape=(max_tokens,))
    x = layers.Dense(hidden_dim, activation="relu")(inputs)
    x = layers.Dense(2*hidden_dim, activation="relu")(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(8, activation="softmax")(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

# word vector input is equal to the vocabulary size
model = get_model(max_tokens=len(vc))
model.summary()


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 23788)]           0         
                                                                 
 dense_4 (Dense)             (None, 16)                380624    
                                                                 
 dense_5 (Dense)             (None, 32)                544       
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_6 (Dense)             (None, 8)                 264       
                                                                 
Total params: 381432 (1.46 MB)
Trainable params: 381432 (1.46 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


#### Train simple model

In [25]:
# cache() is a dataset method that saves the dataset elements in memory or disk and reuses them when called 
callbacks = [ keras.callbacks.ModelCheckpoint("binary_1gram.keras", save_best_only=True) ]
model.fit(binary_1gram_train_ds.cache(), validation_data=binary_1gram_val_ds.cache(), verbose=1, epochs=50, callbacks=callbacks)

Epoch 1/50
908/908 [==============================] - 7s 6ms/step - loss: 1.4792 - accuracy: 0.4978 - val_loss: 1.2672 - val_accuracy: 0.5540
Epoch 2/50
908/908 [==============================] - 9s 10ms/step - loss: 1.2803 - accuracy: 0.5615 - val_loss: 1.1601 - val_accuracy: 0.5963
Epoch 3/50
908/908 [==============================] - 8s 9ms/step - loss: 1.1968 - accuracy: 0.5912 - val_loss: 1.0853 - val_accuracy: 0.6290
Epoch 4/50
908/908 [==============================] - 8s 9ms/step - loss: 1.1459 - accuracy: 0.6138 - val_loss: 1.0340 - val_accuracy: 0.6445
Epoch 5/50
908/908 [==============================] - 8s 9ms/step - loss: 1.1069 - accuracy: 0.6277 - val_loss: 0.9918 - val_accuracy: 0.6636
Epoch 6/50
908/908 [==============================] - 7s 8ms/step - loss: 1.0703 - accuracy: 0.6423 - val_loss: 0.9582 - val_accuracy: 0.6721
Epoch 7/50
908/908 [==============================] - 7s 8ms/step - loss: 1.0442 - accuracy: 0.6539 - val_loss: 0.9298 - val_accuracy: 0.6855
Epoch

#### Accuracy on training data  

In [26]:
# Accuracy on test data
model = keras.models.load_model("binary_1gram.keras")
print(f"Test acc: {model.evaluate(binary_1gram_train_ds)[1]:.3f}")

908/908 [==============================] - 3s 3ms/step - loss: 0.6346 - accuracy: 0.7867
Test acc: 0.787


#### Get the confusion table

In [27]:
pred=model.predict(binary_1gram_train_ds)
print(pred[:3])

predicted = pred.argmax(axis=-1)
print(predicted)

class_names=["C2","C4","C5","C6","C7","C8","C9","C10"]
print(np.array(class_names)[predicted])
print(len(predicted))

908/908 [==============================] - 2s 2ms/step
[[3.2261092e-02 1.5215432e-04 7.3930646e-06 3.3193147e-03 2.1324410e-01
  6.7187923e-01 1.4546755e-03 7.7682048e-02]
 [6.3315651e-04 5.3202332e-04 2.4568506e-05 1.5883227e-01 1.4368439e-01
  2.2029942e-02 1.8594043e-01 4.8832318e-01]
 [3.4681820e-05 3.2450410e-07 3.1985841e-11 6.2369497e-04 9.9891901e-01
  4.0243819e-04 1.4003637e-09 1.9805368e-05]]
[5 7 4 ... 4 4 4]
['C8' 'C10' 'C7' ... 'C7' 'C7' 'C7']
29043


In [28]:
import numpy as np
# This displays the contents of the dataset
# list(int_test_ds)
# Collect targets to use in confusion table
target_arr=[]
for inputs, targets in train_ds:
    target= np.argmax(targets, axis=1)
    target_arr.append(target)

print(target_arr)
    
actual=np.concatenate(target_arr)
print(actual)
print(np.array(class_names)[actual])
cnames=np.array(class_names)[actual]
unique, counts = np.unique(cnames, return_counts=True)
print(unique, counts)

[array([4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 7, 6, 5, 4, 6, 5, 3, 4, 7, 4, 4, 7,
       4, 5, 4, 5, 5, 0, 4, 4, 0, 7]), array([4, 5, 4, 4, 0, 4, 4, 4, 5, 5, 4, 4, 4, 4, 4, 6, 4, 4, 4, 0, 4, 4,
       4, 4, 3, 4, 3, 4, 4, 7, 5, 4]), array([7, 4, 4, 4, 4, 6, 7, 7, 4, 6, 4, 3, 6, 5, 4, 3, 0, 5, 7, 5, 7, 4,
       5, 4, 5, 7, 3, 5, 6, 4, 4, 4]), array([4, 4, 4, 5, 4, 4, 5, 4, 5, 3, 4, 3, 5, 4, 0, 4, 5, 4, 4, 7, 5, 7,
       3, 6, 7, 4, 0, 4, 4, 2, 0, 4]), array([0, 7, 5, 0, 4, 4, 5, 3, 3, 2, 5, 5, 4, 4, 5, 4, 0, 3, 4, 4, 4, 7,
       4, 4, 4, 4, 7, 4, 4, 0, 7, 5]), array([6, 5, 0, 7, 4, 4, 4, 5, 4, 0, 4, 5, 5, 4, 5, 4, 0, 4, 4, 4, 5, 7,
       4, 5, 4, 7, 7, 0, 4, 4, 4, 3]), array([7, 4, 4, 5, 4, 5, 4, 4, 4, 4, 4, 5, 4, 3, 0, 4, 3, 7, 4, 4, 5, 4,
       3, 4, 7, 7, 4, 0, 4, 4, 0, 4]), array([3, 3, 4, 7, 5, 0, 4, 4, 4, 5, 4, 4, 4, 7, 4, 4, 4, 0, 5, 0, 3, 4,
       5, 4, 4, 4, 0, 7, 7, 4, 6, 0]), array([4, 3, 4, 3, 7, 4, 4, 7, 5, 0, 0, 4, 5, 3, 7, 7, 4, 7, 0, 4, 4, 3,
       5, 4, 4, 5, 6, 7, 7, 4,

In [29]:
# show confusion table
# Why confusion table shows different accuracy than accuracy of model evaluate? 
# Need to investigate, but leaving it for later now. 
tf.math.confusion_matrix(actual, predicted)
#tf.math.confusion_matrix(np.array(class_names)[actual], np.array(class_names)[predicted])

<tf.Tensor: shape=(8, 8), dtype=int32, numpy=
array([[ 127,    0,    6,  158, 1311,  344,   44,  303],
       [   1,    0,    0,    4,   20,    2,    0,    2],
       [  11,    0,    0,   15,  121,   35,    6,   30],
       [ 129,    0,    3,  160, 1423,  423,   52,  319],
       [ 726,    0,   30,  879, 7945, 2199,  329, 1798],
       [ 286,    0,    9,  339, 3083,  857,   95,  699],
       [  57,    0,    0,   57,  476,  153,   12,  106],
       [ 195,    0,   11,  256, 2204,  598,   68,  527]], dtype=int32)>

# Word embedding analysis

#### Download GloVe file

!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

#### Parsing the GloVe word-embeddings file

In [11]:
import numpy as np
path_to_glove_file = "glove.6B.100d.txt"
embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print(f"Found {len(embeddings_index)} word vectors.")

Found 400000 word vectors.


#### Preparing the GloVe word-embeddings matrix

In [16]:
embedding_dim = 100
vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))
#print(word_index)

count=0
embedding_matrix = np.zeros((max_tokens, embedding_dim))
for word, i in word_index.items():
    if i < max_tokens:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        count+=1

print(count)

9874


#### GloVe embedding matrix misses words. 

# Continue to explore other approackes
#### Train model with embedding

## Different output mode -  integer sequence datasets
To use embedding we need fixed sequence length. Therefore, it is better and more interesting to have keywords from separate papers separately in this case. Each country must have country_article_no.txt. For this dataset was transformed. 

##### Later try to classify countries .

In [30]:
import os, pathlib, shutil, random
import tensorflow as tf
from tensorflow import keras

batch_size = 100
train_ds = keras.utils.text_dataset_from_directory("kwdsuic/kwd_per_paper/train", batch_size=batch_size, label_mode="categorical", shuffle=True,
                                                  labels="inferred", class_names=["C2","C4","C5","C6","C7","C8","C9","C10"] )

#val_ds = keras.utils.text_dataset_from_directory("kwdsuic/train", batch_size=batch_size, label_mode="categorical")
#test_ds = keras.utils.text_dataset_from_directory("kwdsuic/train", batch_size=batch_size, label_mode="categorical")

Found 29043 files belonging to 8 classes.


In [31]:
# This function returns only data part without target 
# to create a new dataset that will be used to create dictionary
text_only_train_ds = train_ds.map(lambda x, y: x)
for inputs in text_only_train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("inputs[0]:", inputs[0])
    break


inputs.shape: (100,)
inputs.dtype: <dtype: 'string'>
inputs[0]: tf.Tensor(b'pre_messenger_rna 5_ht2c_receptor protein depression nerve mice activation suicide injury brain', shape=(), dtype=string)


In [32]:
import tensorflow as tf
from tensorflow.keras import layers

# max_length is a length of the vector that encodes text
# 250 size gave ~0.85 on test data
# 600 with 4 LSTM units does not train at all
# the reviews are about 300 words
max_length = 30 
max_tokens = 30000
text_vectorization = layers.TextVectorization( max_tokens=max_tokens, output_mode="int", output_sequence_length=max_length)

text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_val_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_test_ds = train_ds.map(lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

In [33]:
for inputs, targets in int_train_ds:
    print(inputs.shape)
    print(inputs[0])
    print(targets[0])
    # With this test_input variable verify tf.one_hot() transformation
    test_input=inputs[0]
    break

(100, 30)
tf.Tensor(
[  33  621 1839  182    8   91   12    7   72 1153    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0], shape=(30,), dtype=int64)
tf.Tensor([0. 0. 0. 1. 0. 0. 0. 0.], shape=(8,), dtype=float32)


In [34]:
vc=text_vectorization.get_vocabulary()
for tg in vc:
    if 'sexual' in tg:
        print(tg)

print(len(vc))

sexualabuse
sexualorientation
childhoodsexualabuse
sexualviolence
sexualminority
sexualbehavior
homosexuality
sexuality
militarysexualtrauma
bisexualmen
bisexual
sexualassault
childsexualabuse
bisexualadults
sexualhealth
bisexualyouth
sexualtrauma
transsexual
sexuallytransmittedinfections
sexualminorityyouth
sexuallytransmitteddiseases
sexualandgenderminorities
sexualbehaviors
bisexualyouths
sexualminorities
sexualharassment
transsexuals
sexualidentity
sexualdysfunction
sexualandreproductivehealthandrights
hypersexuality
bisexualpopulations
bisexuality
transsexuality
transsexualism
sexualrisktaking
sexualriskbehaviors
sexualriskbehavior
sexualrisk
sexuallytransmitteddisease
riskysexualbehaviors
homosexualmen
homosexual
heterosexualyouth
bisexuals
adolescentsexualorientation
sociosexuality
sexualvictimization
sexualstigma
sexualorientationdisparities
sexualintercourse
sexualhomicide
sexualhealthpromotion
sexualcompulsivity
sexualbehaviour
sexualaddiction
sexualactivity
sexsexualpartners

#### Try RNN for text classification using integer feature vector

In [39]:
hidden_dim = 16
# The sizes of layers are minimized in order to run
import tensorflow as tf

inputs = keras.Input(shape=(None,), dtype="int64")
# Here will be binary input as test_input
embedded = tf.one_hot(inputs, depth=max_tokens)
print(embedded)

# RNN layer 
# NOT USE TOO BIG MODEL : x = layers.Bidirectional(layers.LSTM(32))(embedded)
# When units=4 the model runs. 
x = layers.LSTM(10)(embedded)

x = layers.Dense(hidden_dim, activation="relu")(x)
x = layers.Dense(2*hidden_dim, activation="relu")(x)

x = layers.Dropout(0.1)(x)
outputs = layers.Dense(8, activation="softmax")(x)

model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()


KerasTensor(type_spec=TensorSpec(shape=(None, None, 30000), dtype=tf.float32, name=None), name='tf.one_hot_2/one_hot:0', description="created by layer 'tf.one_hot_2'")
Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, None)]            0         
                                                                 
 tf.one_hot_2 (TFOpLambda)   (None, None, 30000)       0         
                                                                 
 lstm_2 (LSTM)               (None, 10)                1200440   
                                                                 
 dense_8 (Dense)             (None, 16)                176       
                                                                 
 dense_9 (Dense)             (None, 32)                544       
                                                                 
 dropout_4 (Dropout)   

In [40]:
callbacks = [ keras.callbacks.ModelCheckpoint("int_lstm.keras", save_best_only=True) ]

In [43]:
model.fit(int_train_ds, validation_data=int_val_ds, epochs=100, verbose=1, callbacks=callbacks)

Epoch 1/100
291/291 [==============================] - 11s 34ms/step - loss: 1.1761 - accuracy: 0.5920 - val_loss: 1.1214 - val_accuracy: 0.6098
Epoch 2/100
291/291 [==============================] - 10s 34ms/step - loss: 1.1607 - accuracy: 0.5980 - val_loss: 1.1316 - val_accuracy: 0.6048
Epoch 3/100
291/291 [==============================] - 10s 34ms/step - loss: 1.1469 - accuracy: 0.6031 - val_loss: 1.0981 - val_accuracy: 0.6163
Epoch 4/100
291/291 [==============================] - 10s 34ms/step - loss: 1.1340 - accuracy: 0.6073 - val_loss: 1.1017 - val_accuracy: 0.6178
Epoch 5/100
291/291 [==============================] - 10s 34ms/step - loss: 1.1236 - accuracy: 0.6097 - val_loss: 1.0744 - val_accuracy: 0.6246
Epoch 6/100
291/291 [==============================] - 10s 34ms/step - loss: 1.1116 - accuracy: 0.6137 - val_loss: 1.0640 - val_accuracy: 0.6262
Epoch 7/100
291/291 [==============================] - 10s 34ms/step - loss: 1.1008 - accuracy: 0.6171 - val_loss: 1.0577 - val_ac

In [44]:
model = keras.models.load_model("int_lstm.keras")
print(f"Test acc: {model.evaluate(int_train_ds)[1]:.3f}")

291/291 [==============================] - 5s 14ms/step - loss: 0.6770 - accuracy: 0.7658
Test acc: 0.766


#### Try to use Enbedding layer for the previous problem 

In [59]:
vocab_size = len(vc)
embed_dim = 256
hidden_dim = 100
max_length = 30
batch_size=32

# ignore from FC book Ch.11 Listing 11.22
#num_heads = 2
#dense_dim = 32

inputs = keras.Input(shape=(max_length), dtype="int64")

embedded = layers.Embedding(vocab_size, embed_dim, input_length=max_length, name="embedded" )(inputs)

x = layers.Bidirectional(layers.LSTM(32))(embedded)
#x = layers.LSTM(32)(embedded)

#x = layers.Dense(hidden_dim, activation="relu")(x)
#x = layers.Dense(2*hidden_dim, activation="relu")(x)

x = layers.Dropout(0.1)(x)
outputs = layers.Dense(8, activation="softmax")(x)

model = keras.Model(inputs, outputs)

# Maybe optimizer could be different? 
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 30)]              0         
                                                                 
 embedded (Embedding)        (None, 30, 256)           6089984   
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                73984     
 onal)                                                           
                                                                 
 dropout_10 (Dropout)        (None, 64)                0         
                                                                 
 dense_22 (Dense)            (None, 8)                 520       
                                                                 
Total params: 6164488 (23.52 MB)
Trainable params: 6164488 (23.52 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [60]:
callbacks = [ keras.callbacks.ModelCheckpoint("int_embed_lstm.keras", save_best_only=True) ]
model.fit(int_train_ds, validation_data=int_train_ds, epochs=50, verbose=1, callbacks=callbacks)

Epoch 1/50
291/291 [==============================] - 11s 30ms/step - loss: 1.4733 - accuracy: 0.4862 - val_loss: 1.3527 - val_accuracy: 0.5155
Epoch 2/50
291/291 [==============================] - 3s 12ms/step - loss: 1.3061 - accuracy: 0.5462 - val_loss: 1.1873 - val_accuracy: 0.5804
Epoch 3/50
291/291 [==============================] - 3s 11ms/step - loss: 1.1810 - accuracy: 0.5888 - val_loss: 1.0454 - val_accuracy: 0.6413
Epoch 4/50
291/291 [==============================] - 3s 10ms/step - loss: 1.0729 - accuracy: 0.6303 - val_loss: 0.9486 - val_accuracy: 0.6780
Epoch 5/50
291/291 [==============================] - 3s 9ms/step - loss: 0.9825 - accuracy: 0.6616 - val_loss: 0.8557 - val_accuracy: 0.7035
Epoch 6/50
291/291 [==============================] - 3s 10ms/step - loss: 0.9102 - accuracy: 0.6862 - val_loss: 0.7954 - val_accuracy: 0.7260
Epoch 7/50
291/291 [==============================] - 3s 11ms/step - loss: 0.8514 - accuracy: 0.7048 - val_loss: 0.7601 - val_accuracy: 0.7417

In [ ]:
model = keras.models.load_model("int_embed_lstm.keras")
print(f"Test acc: {model.evaluate(int_train_ds)[1]:.3f}")

## Next explore the embedding layer

In [79]:
23789/(30*256)

3.097526041666667

In [120]:
import pandas as pd
# how to get the layer
l=model.get_layer("embedded").output
print(l)
w=model.get_layer("embedded").get_weights()
for i in range(23787,23789):
    print(i)
    print(len(w[0][i] ) )
    
#print(len(w[0][1] ) )

#wdf=pd.DataFrame(w)
#vc=text_vectorization.get_vocabulary()
#for tg in vc:
#    print(tg)

print(len(vc))

KerasTensor(type_spec=TensorSpec(shape=(None, 30, 256), dtype=tf.float32, name=None), name='embedded/embedding_lookup/Identity:0', description="created by layer 'embedded'")


InternalError: Could not synchronize CUDA stream: CUDA_ERROR_LAUNCH_FAILED: unspecified launch failure

In [119]:
import numpy as np

for inputs, targets in int_train_ds:
    print(inputs.shape)
    print(inputs[0])
    print(targets[0])
    # With this test_input variable verify tf.one_hot() transformation
    break
p=model.predict(inputs)
print(p[0])
print(targets[0])

for i in range(100):
    ind=[x for x in inputs[i].numpy() if x >0]
    words=[vc[i] for i in ind]
    print(ind)
    print(words)
    print(np.argmax(p[i]) , np.argmax(targets[i]) )
#target= np.argmax(targets, axis=1)

(100, 30)
tf.Tensor(
[  705   925   602  3517   309    42  1102 19535   104    26     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0], shape=(30,), dtype=int64)
tf.Tensor([0. 0. 0. 0. 1. 0. 0. 0.], shape=(8,), dtype=float32)
4/4 [==============================] - 0s 3ms/step
[2.7696323e-02 1.6566212e-06 2.1807733e-04 4.7049302e-04 9.7160000e-01
 1.1792747e-05 4.7009894e-07 1.1961646e-06]
tf.Tensor([0. 0. 0. 0. 1. 0. 0. 0.], shape=(8,), dtype=float32)
[705, 925, 602, 3517, 309, 42, 1102, 19535, 104, 26]
['deficithyperactivitydisorder', 'attentiondeficithyperactivitydisorder', 'worldhealthorganization', 'reportscaleasrs', 'psychiatriccomorbidity', 'followup', 'representativesample', 'familialassociation', 'adolescentsuicide', 'mentaldisorders']
4 4
[3601, 23357, 15214]
['parentaldivorce', 'adultsuicideattempt', 'parentalalcoholabuse']
4 4
[2, 158, 1025, 3291, 505, 1339, 1465]
['suicide', 'smoking', 'nicotine', 'cotini

### The more complex architecture the poorer results are

#### Model that uses Encoder and MultiHeadAttention layers ( by some analogy to Listing 11.22 )

In [160]:
# What can we do to improve? Here we dont have any RNN, just transformations 

vocab_size = 24000
embed_dim = 100

# from FC book Ch.11 Listing 11.22
# num_heads = 2 - acheved 8.69
# Lets try more heads/words
# With 4 heads 8.73 , just a bit , with 5 heads 8.70
# Still worse than a simpler method. 
num_heads = 5
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)

# try to add attention 
x = layers.MultiHeadAttention( num_heads=num_heads, key_dim=embed_dim)(x,x)

# This layer is needed - what it does? 
x = layers.GlobalMaxPooling1D()(x)

x = layers.Dropout(0.5)(x)
outputs = layers.Dense(8, activation="softmax")(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

Model: "model_21"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_20 (InputLayer)       [(None, None)]               0         []                            
                                                                                                  
 embedding_7 (Embedding)     (None, None, 100)            2400000   ['input_20[0][0]']            
                                                                                                  
 multi_head_attention (Mult  (None, None, 100)            201600    ['embedding_7[0][0]',         
 iHeadAttention)                                                     'embedding_7[0][0]']         
                                                                                                  
 global_max_pooling1d (Glob  (None, 100)                  0         ['multi_head_attention[

#### Use model with attention , it really improved from previous, but not compared to more simple model of 2 Ngram.  

In [161]:
callbacks = [ keras.callbacks.ModelCheckpoint("attention.keras", save_best_only=True) ]
model.fit(int_train_ds, validation_data=int_train_ds, epochs=10, callbacks=callbacks)

Epoch 1/10
908/908 [==============================] - 23s 24ms/step - loss: 0.2124 - accuracy: 0.9586 - val_loss: 8.5312 - val_accuracy: 0.0296
Epoch 2/10
908/908 [==============================] - 22s 24ms/step - loss: 0.1551 - accuracy: 0.9736 - val_loss: 9.4029 - val_accuracy: 0.0296
Epoch 3/10
908/908 [==============================] - 22s 25ms/step - loss: 0.1216 - accuracy: 0.9822 - val_loss: 10.5073 - val_accuracy: 0.0296
Epoch 4/10
908/908 [==============================] - 23s 25ms/step - loss: 0.1143 - accuracy: 0.9846 - val_loss: 11.8767 - val_accuracy: 0.0296
Epoch 5/10
908/908 [==============================] - 22s 25ms/step - loss: 0.1200 - accuracy: 0.9847 - val_loss: 12.7075 - val_accuracy: 0.0296
Epoch 6/10
908/908 [==============================] - 23s 25ms/step - loss: 0.1209 - accuracy: 0.9848 - val_loss: 12.9130 - val_accuracy: 0.0296
Epoch 7/10
908/908 [==============================] - 23s 25ms/step - loss: 0.1343 - accuracy: 0.9836 - val_loss: 13.3451 - val_accu

In [162]:
model = keras.models.load_model("attention.keras")
print(f"Test acc: {model.evaluate(int_train_ds)[1]:.3f}")

908/908 [==============================] - 6s 7ms/step - loss: 8.5312 - accuracy: 0.0296
Test acc: 0.030
